In [43]:
! dvc init


You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|              https://dvc.org/doc/user-guide/analytics               |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: https://dvc.org/doc
- Get help and share ideas: https://dvc.org/chat
- Star us on GitHub: https://github.com/iterative/dvc


## Load data

In [44]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [45]:
! dvc get https://github.com/iterative/dataset-registry \
          get-started/data.xml -o data/data.xml
! dvc add data/data.xml

0% Downloading|                                    |0/1 [00:00<?,     ?file/s]
!
  0%|          |get-started/data.xml           0.00/37.9M [00:00<?,       ?it/s]
  0%|          |get-started/data.xml      64.0k/36.1M [00:00<02:49,     223kB/s]
  0%|          |get-started/data.xml       128k/36.1M [00:00<02:23,     263kB/s]
  1%|          |get-started/data.xml       256k/36.1M [00:00<02:04,     301kB/s]
  2%|▏         |get-started/data.xml       576k/36.1M [00:00<01:32,     405kB/s]
  2%|▏         |get-started/data.xml       704k/36.1M [00:01<01:18,     476kB/s]
  2%|▏         |get-started/data.xml       896k/36.1M [00:01<01:01,     597kB/s]
  3%|▎         |get-started/data.xml      1.00M/36.1M [00:01<00:55,     662kB/s]
  3%|▎         |get-started/data.xml      1.12M/36.1M [00:01<00:50,     725kB/s]
  3%|▎         |get-started/data.xml      1.25M/36.1M [00:01<00:47,     767kB/s]
  4%|▍         |get-started/data.xml      1.38M/36.1M [00:01<00:45,     809kB/s]
  4%|▍         |get-started/

## Create a pipeline with some steps. i.e.:
    *  prepare data
    *  turn data into features
    *  train models from features
    *  evaluate models

In [46]:
! dvc run -f -n prepare \
             -p prepare.seed,prepare.split \
             -d src/prepare.py -d data/data.xml \
             -o data/prepared \
             python src/prepare.py data/data.xml

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Running stage 'prepare' with command:
	python src/prepare.py data/data.xml
Creating 'dvc.yaml'
Adding stage 'prepare' in 'dvc.yaml'
Generating lock file 'dvc.lock'

To track the changes with git, run:

	git add data/.gitignore dvc.yaml dvc.lock


In [47]:
! dvc run -f -n featurize \
          -p featurize.max_features,featurize.ngrams \
          -d src/featurization.py -d data/prepared \
          -o data/features \
          python src/featurization.py data/prepared data/features

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Running stage 'featurize' with command:
	python src/featurization.py data/prepared data/features
The input data frame data/prepared/train.tsv size is (20017, 3)
The output matrix data/features/train.pkl size is (20017, 502) and data type is float64
The input data frame data/prepared/test.tsv size is (4983, 3)
The output matrix data/features/test.pkl size is (4983, 502) and data type is float64
Adding stage 'featurize' in 'dvc.yaml'
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock data/.gitignore dvc.yaml


In [48]:
! dvc run -f -n train \
          -p train.seed,train.n_estimators \
          -d src/train.py -d data/features \
          -o model.pkl \
          python src/train.py data/features model.pkl

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Running stage 'train' with command:
	python src/train.py data/features model.pkl
Input matrix size (20017, 502)
X matrix size (20017, 500)
Y matrix size (20017,)
If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Adding stage 'train' in 'dvc.yaml'
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.yaml dvc.lock


In [49]:
! dvc run -f -n evaluate \
          -d src/evaluate.py -d model.pkl -d data/features \
          -M scores.json \
          --plots-no-cache prc.json \
          python src/evaluate.py model.pkl \
                 data/features scores.json prc.json

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Running stage 'evaluate' with command:
	python src/evaluate.py model.pkl data/features scores.json prc.json
If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Adding stage 'evaluate' in 'dvc.yaml'
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.yaml dvc.lock


In [50]:
! dvc repro

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Stage 'data/data.xml.dvc' didn't change, skipping
Stage 'prepare' didn't change, skipping
Stage 'featurize' didn't change, skipping
Stage 'train' didn't change, skipping
Stage 'evaluate' didn't change, skipping
Data and pipelines are up to date.


In [51]:
! dvc remote add -df myremote gs://dvc_intro
! git add .
! git commit -m 'run'
! git push origin data_pipelines

Setting 'myremote' as a default remote.
[data_pipelines 70bd6eb] run
 12 files changed, 653 insertions(+), 408 deletions(-)
 create mode 100644 .dvc/.gitignore
 create mode 100644 .dvc/config
 create mode 100644 .dvc/plots/confusion.json
 create mode 100644 .dvc/plots/default.json
 create mode 100644 .dvc/plots/scatter.json
 create mode 100644 .dvc/plots/smooth.json
 create mode 100644 data/.gitignore
 create mode 100644 data/data.xml.dvc
 create mode 100644 dvc.lock
 create mode 100644 dvc.yaml
 create mode 100644 prc.json
 rewrite test.ipynb (76%)
Enumerating objects: 13, done.
Counting objects: 100% (13/13), done.
Delta compression using up to 8 threads
Compressing objects: 100% (8/8), done.
Writing objects: 100% (9/9), 7.33 KiB | 2.44 MiB/s, done.
Total 9 (delta 5), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (5/5), completed with 3 local objects.
To github.com:NichitaDiaconu/dvc_intro.git
   d86de9b..70bd6eb  data_pipelines -> data_pipelines


In [52]:
! cat scores.json

{"auc": 0.5417487597055675}

In [53]:
! dvc push

0% Uploading|                                      |0/1 [00:00<?,     ?file/s]
!
  0%|          |data/data.xml                  0.00/37.9M [00:00<?,       ?it/s]
 28%|██▊       |data/data.xml             10.0M/36.1M [00:00<00:00,    45.9MB/s]
 55%|█████▌    |data/data.xml             20.0M/36.1M [00:02<00:01,    12.3MB/s]
 83%|████████▎ |data/data.xml             30.0M/36.1M [00:04<00:00,    8.37MB/s]
100%|██████████|data/data.xml             36.1M/36.1M [00:06<00:00,    5.89MB/s]
1 file pushed


In [54]:
! dvc dag

7=






















    +-------------------+  
    | data/data.xml.dvc |  
    +-------------------+  
              *            
              *            
              *            
         +---------+       
         | prepare |       
         +---------+       
              *            
              *            
              *            
        +-----------+      
        | featurize |      
        +-----------+      
         **        **      
       **            *     
      *               **   
+-------+               *  
| train |             **   
+-------+            *     
         **        **      
           **    **        
:

KeyboardInterrupt: 

In [55]:
! rm -rf data/features
! rm -rf data/prepared
! rm -rf data/data.xml
! rm -rf model.pkl
! rm -rf .dvc/cache
! rm -rf .dvc/tmp

### We can have as many pipeline steps that we want
### Each can be computed sequentially on separate machines and what not